## Librerias

In [82]:
from IPython.display import display, Markdown
import pandas as pd
import random
from datetime import datetime, timedelta
from typing import List, Tuple

In [64]:
!pip install faker
import faker
fake = faker.Faker()

In [65]:
!pip install psycopg2-binary
!pip install sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

## Coneccion a la base de datos

In [66]:
# Connect to the database and populate with fake data

host = "localhost"
port = 5432
user = "catedra"
password = "S3cret"
database = "postgres"
driver = "postgresql"

pg = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database))

Session = sessionmaker(bind=pg)

## Mercadopago.SQL

### Correr desde archivo

In [91]:
# correr mercadopago.sql
with open("mercadopago.sql") as f:
    sql = f.read()
    pg.execute(text(sql))

### Correr desde string

#### Vaciar la base de datos

In [67]:
delete_all = """
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;
GRANT ALL ON SCHEMA public TO public;
"""

pg.execute(delete_all)

#### Crear Tablas en la base de datos

In [68]:
create_tables = """
CREATE TABLE Clave (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    alias VARCHAR(50) NOT NULL,
    esVirtual BOOLEAN NOT NULL
);

CREATE TABLE Usuarios (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    CUIT VARCHAR(50),
    email VARCHAR(50),
    nombre VARCHAR(50),
    apellido VARCHAR(50),
    username VARCHAR(50),
    password VARCHAR(50),
    saldo FLOAT,
    fecha_alta DATE,
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE CuentaBancaria (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    banco VARCHAR(50),
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE ProveedorServicio (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    nombre_empresa VARCHAR(50),
    categoria_servicio VARCHAR(50),
    fecha_alta DATE,
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE Tarjeta (
    numero VARCHAR(50) PRIMARY KEY,
    vencimiento DATE,
    cvv INTEGER,
    CU VARCHAR(50),
    FOREIGN KEY (CU) REFERENCES Clave(clave_uniforme),
    CHECK (CU IS NOT NULL)
);

-- Add trigger on insert to check if cu is virtual, if not, raise exception

CREATE OR REPLACE FUNCTION check_cu_virtual() 
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT esVirtual FROM Clave WHERE clave_uniforme = NEW.CU) = FALSE THEN
        RAISE EXCEPTION 'CU is not virtual';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_cu_virtual_trigger
BEFORE INSERT ON Tarjeta
FOR EACH ROW
EXECUTE FUNCTION check_cu_virtual();

CREATE TABLE Transaccion (
    codigo SERIAL PRIMARY KEY,
    CU_Origen VARCHAR(50),
    CU_Destino VARCHAR(50),
    monto FLOAT,
    fecha DATE,
    descripcion VARCHAR(50),
    estado VARCHAR(50),
    es_con_tarjeta BOOLEAN,
    numero VARCHAR(50),
    interes FLOAT,
    FOREIGN KEY (CU_Origen) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (CU_Destino) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (numero) REFERENCES Tarjeta(numero)
);

-- Create trigger to check if user has enough balance to make transaction without card
CREATE OR REPLACE FUNCTION check_balance()
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT saldo FROM Usuarios WHERE clave_uniforme = NEW.CU_Origen) < NEW.monto THEN
        RAISE EXCEPTION 'Not enough balance';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_balance_trigger
BEFORE INSERT ON Transaccion
FOR EACH ROW
WHEN (NEW.es_con_tarjeta = FALSE)
EXECUTE FUNCTION check_balance();


CREATE TABLE Rendimiento (
    id SERIAL PRIMARY KEY,
    fecha_pago DATE,
    comienzo_plazo DATE,
    fin_plazo DATE,
    TNA FLOAT,
    monto FLOAT
);

CREATE TABLE RendimientoUsuario (
    clave_uniforme VARCHAR(50),
    id INTEGER,
    PRIMARY KEY (clave_uniforme, id),
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (id) REFERENCES Rendimiento(id)
);

CREATE TABLE TransaccionTarjeta (
    codigo INTEGER,
    numero VARCHAR(50),
    PRIMARY KEY (codigo, numero),
    FOREIGN KEY (numero) REFERENCES Tarjeta(numero),
    FOREIGN KEY (codigo) REFERENCES Transaccion(codigo)
);

"""

pg.execute(create_tables)

## Crear datos de prueba

In [151]:
"""

Alias CBU
En agosto de 2016, el BCRA creó la funcionalidad denominada "Alias CBU", la cual permite al cliente bancario incorporar un alias a cada una de sus CBUs, desde la opción de "Administración de cuentas de transferencias" disponible en banca por Internet y en banca móvil.3​

El Alias CBU tiene entre 6 y 20 caracteres con letras, números y símbolos (sólo punto y guion del medio), como "alejandragisela20″, "clara-delcarril". Es único e irrepetible para todo el sistema financiero, mientras que el CBU no desaparece ya que los 22 números respaldan a ese Alias.3​4​5​

En abril de 2017, el BCRA emitió la comunicación A6215 donde expresó que "...asignará un Alias para cada uno de las CBU que, al 1/6/2017, no tengan registrado el mismo en el reservorio que se encuentra a su cargo",6​ surgiendo dichos alias de forma automática a través de un algoritmo que combina tres palabras simples del castellano.6​7​

Estructura de la CBU

Estructura de una Clave Bancaria Uniforme
La CBU está compuesta por 22 dígitos, separados en dos bloques. El primer bloque tiene un número de entidad de 3 dígitos, un número de sucursal de 4 dígitos y un dígito verificador. El segundo bloque tiene un número de 13 dígitos que identifica la cuenta dentro de la entidad y la sucursal, más un dígito verificador.

Para el número de entidad de 3 dígitos que compone el primer bloque, debe verificarse el código de banco con la siguiente tabla de bancos

005	The Royal Bank of Scotland N.V.
007	Banco de Galicia14 y Buenos Aires S.A.
011	Banco de la Nación Argentina
014	Banco de la Provincia de Buenos Aires
015	Industrial and Commercial Bank of China S.A.
016	Citibank N.A.
017	BBvA Banco Francés S.A.
018	The Bank of Tokyo-Mitsubishi UFJ, LTD.
020	Banco de la Provincia de Córdoba S.A.
027	Banco Supervielle S.A.
029	Banco de la Ciudad de Buenos Aires
030	Central de la República Argentina
034	Banco Patagonia S.A.
044	Banco Hipotecario S.A.
045	Banco de San Juan S.A.
046	Banco do Brasil S.A.
060	Banco de Tucumán S.A.
065	Banco Municipal de Rosario
072	Banco Santander Río S.A.
083	Banco del Chubut S.A.
086	Banco de Santa Cruz S.A.
093	Banco de la Pampa Sociedad de Economía Mixta
094	Banco de Corrientes S.A.
097	Banco Provincia del Neuquén S.A.
143	Brubank S.A.U.
147	Banco Interfinanzas S.A.
150	HSBC Bank Argentina S.A.
158	Openbank
165	JP Morgan Chase Bank NA (Sucursal Buenos Aires)
191	Banco Credicoop Cooperativo Limitado
198	Banco de Valores S.A.
247	Banco Roela S.A.
254	Banco Mariva S.A.ī
259	Banco Itaú Argentina S.A.
262	Bank of America National Association
266	BNP Paribas
268	Banco Provincia de Tierra del Fuego
269	Banco de la República Oriental del Uruguay
277	Banco Sáenz S.A.
281	Banco Meridian S.A.
285	Banco Macro S.A.
295	American Express Bank LTD. S.A.
299	Banco Comafi S.A.
300	Banco de Inversión y Comercio Exterior S.A.
301	Banco Piano S.A.
305	Banco Julio S.A.
309	Nuevo Banco de la Rioja S.A.
310	Banco del Sol S.A.
311	Nuevo Banco del Chaco S.A.
312	MBA Lazard Banco de Inversiones S.A.
315	Banco de Formosa S.A.
319	Banco CMF S.A.
321	Banco de Santiago del Estero S.A.
322	Banco Industrial S.A.
325	Deutsche Bank S.A.
330	Nuevo Banco de Santa Fe S.A.
331	Banco Cetelem Argentina S.A.
332	Banco de Servicios Financieros S.A.
336	Banco Bradesco Argentina S.A.
338	Banco de Servicios y Transacciones S.A.
339	RCI Banque S.A.
340	BACS Banco de Crédito y Securitización S.A.
341	Más Ventas S.A.
384	Wilobank S.A.
386	Nuevo Banco de Entre Ríos S.A.
389	Banco Columbia S.A.
405	Ford Credit Compañía Financiera S.A.
406	Metrópolis Compañía Financiera S.A.
408	Compañía Financiera Argentina S.A.
413	Montemar Compañía Financiera S.A.
415	Transatlántica Compañía Financiera S.A.
428	Caja de Crédito Coop. La Capital del Plata LTDA.
431	Banco Coinag S.A.
432	Banco de Comercio S.A.
434	Caja de Crédito Cuenca Coop. LTDA.
437	Volkswagen Credit Compañía Financiera S.A.
438	Cordial Compañía Financiera S.A.
440	Fiat Crédito Compañía Financiera S.A.
441	GPAT Compañía Financiera S.A.
442	Mercedes-Benz Compañía Financiera Argentina S.A.
443	Rombo Compañía Financiera S.A.
444	John Deere Credit Compañía Financiera S.A.
445	PSA Finance Argentina Compañía Financiera S.A.
446	Toyota Compañía Financiera de Argentina S.A.
448	Finandino Compañía Financiera S.A.
453	Naranja X
992	Provincanje S.A.

La Clave Única de Identificación Tributaria (CUIT) es un código con el que la AFIP identifica a trabajadores autónomos, comercios y empresas.

Se compone de 11 dígitos:

un prefijo de 2 dígitos aleatorios, de carácter genérico y no binario en términos de género.

seguido por 8 dígitos que corresponden, en el caso de personas humanas, al número de Documento Nacional de Identidad (DNI), y en el caso de empresas al número de sociedad asignado.

1 dígito verificador.

En el caso de las personas humanas coincidirá con el Código de Identificación Laboral (CUIL) otorgado por la Administración Nacional de la Seguridad Social (ANSES).

Fuente: AFIP
"""


'\n\nAlias CBU\nEn agosto de 2016, el BCRA creó la funcionalidad denominada "Alias CBU", la cual permite al cliente bancario incorporar un alias a cada una de sus CBUs, desde la opción de "Administración de cuentas de transferencias" disponible en banca por Internet y en banca móvil.3\u200b\n\nEl Alias CBU tiene entre 6 y 20 caracteres con letras, números y símbolos (sólo punto y guion del medio), como "alejandragisela20″, "clara-delcarril". Es único e irrepetible para todo el sistema financiero, mientras que el CBU no desaparece ya que los 22 números respaldan a ese Alias.3\u200b4\u200b5\u200b\n\nEn abril de 2017, el BCRA emitió la comunicación A6215 donde expresó que "...asignará un Alias para cada uno de las CBU que, al 1/6/2017, no tengan registrado el mismo en el reservorio que se encuentra a su cargo",6\u200b surgiendo dichos alias de forma automática a través de un algoritmo que combina tres palabras simples del castellano.6\u200b7\u200b\n\nEstructura de la CBU\n\nEstructura de 

In [152]:
def random_dni():
    return str(random.randint(10**7, 10**8 - 1))


def random_cuit():
    return (
        str(random.randint(10, 99))
        + "-"
        + random_dni()
        + "-"
        + str(random.randint(0, 9))
    )


def random_cu():
    return (
        "0"
        + str(random.randint(0, 999))
        + "0"
        + str(random.randint(0, 9999))
        + str(random.randint(0, 9))
        + str(random.randint(10**12, 10**13 - 1))
    )


def random_alias():
    return ".".join(fake.words(nb=3))


cu = random_cu()
alias = random_alias()
cuit = random_cuit()

display(cu, alias, cuit)


'02470307052900979637458'

'perform.find.black'

'43-87490041-9'

In [153]:
def insert_clave_uniforme(
        cu: str,
        alias: str,
        esVirtual: bool
    ) -> None:
    """Inserta una clave uniforme en la base de datos

    Args:
        cu (str): Clave Uniforme
        alias (str): Alias
        esVirtual (bool): Es virtual o no
    """

    pg.execute("INSERT INTO Clave VALUES (%s, %s, %s)", (cu, alias, esVirtual))


def create_identifiers() -> Tuple[str, str, str]:
    """Crea identificadores aleatorios

    Returns:
        Tuple[str, str, str]: CU, Alias, CUIT
    """

    cu = random_cu()
    alias = random_alias()
    cuit = random_cuit()

    return cu, alias, cuit


def create_usuario() -> Tuple[str, str, str, str, str, str, str, str]:
    """Crea un usuario aleatorio

    Returns:
        Tuple[str, str, str, str, str, str, str, str]: CVU, Alias, CUIT, Email, Nombre, Apellido, Username, Password
    """

    cvu, alias, cuit = create_identifiers()
    insert_clave_uniforme(cvu, alias, True)
    pg.execute(
        "INSERT INTO Usuarios VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (
            cvu,
            cuit,
            (email := fake.email()),
            (first_name := fake.first_name()),
            (last_name := fake.last_name()),
            (user_name := fake.user_name()),
            (password := fake.password()),
            0,
            fake.date_this_century(),
        ),
    )
    return cvu, alias, cuit, email, first_name, last_name, user_name, password


def create_cuenta_bancaria() -> Tuple[str, str]:
    """Crea una cuenta bancaria aleatoria

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute("INSERT INTO CuentaBancaria VALUES (%s, %s)", (cvu, fake.company()))
    return cvu, alias


def create_proveedor_servicio() -> Tuple[str, str]:
    """Crea un proveedor de servicio aleatorio

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute(
        "INSERT INTO ProveedorServicio VALUES (%s, %s, %s, %s)",
        (cvu, fake.company(), fake.word(), fake.date_this_century()),
    )
    return cvu, alias


def add_tarjeta(
        cvu : str,
        number : str,
        cvv : str,
        vencimiento : str
    ):
    """Agrega una tarjeta a la base de datos

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta
        number (str): Número de tarjeta
        cvv (str): CVV
        vencimiento (str): Fecha de vencimiento
    """

    pg.execute(
        "INSERT INTO Tarjeta VALUES (%s, %s, %s, %s)", (number, vencimiento, cvv, cvu)
    )


def create_tarjeta(cvu : str) -> Tuple[str, str, str]:
    """Crea una tarjeta aleatoria

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta

    Returns:
        Tuple[str, str, str]: Número, CVV, Vencimiento
    """
    
    add_tarjeta(
        cvu,
        (num := fake.credit_card_number()),
        (cvv := fake.credit_card_security_code()),
        (vencimiento := fake.date_between(start_date="-1y", end_date="+1y")),
    )
    return num, cvv, vencimiento


cvu1, alias1, cuit1, email1, first_name1, last_name1, user_name1, password1 = create_usuario()
cvu2, alias2, cuit2, email2, first_name2, last_name2, user_name2, password2 = create_usuario()

cvu_serv1, alias_serv1 = create_proveedor_servicio()
cvu_cuenta1, alias_cuenta1 = create_cuenta_bancaria()

display(
    Markdown(
        f"""
# Usuarios
- {first_name1} {last_name1} - {email1} - {user_name1} - {password1} | {cuit1} - {cvu1} - {alias1}
- {first_name2} {last_name2} - {email2} - {user_name2} - {password2} | {cuit2} - {cvu2} - {alias2}

# Proveedores de Servicio
- {alias_serv1} - {cvu_serv1}

# Cuentas Bancarias
- {alias_cuenta1} - {cvu_cuenta1}
"""
    )

)


# Usuarios
- Jared Chandler - iford@example.com - greerryan - V(x91Es^lH | 41-38366181-8 - 04830279907429520135534 - enough.relate.understand
- Anthony Jones - james48@example.org - showe - IB9WTWsmC* | 74-59695361-6 - 05140685864015702391984 - record.course.room

# Proveedores de Servicio
- evidence.cut.enter - 04260777136730688130505

# Cuentas Bancarias
- get.heavy.staff - 0780665243583035456540


In [154]:
num_tar1, cvv_tar1, venc_tar1 = create_tarjeta(cvu1)
num_tar1, cvv_tar1, venc_tar1

('2237430911434611', '4079', datetime.date(2025, 4, 3))

In [155]:
# Try to insert a transaction without enough balance

try:
    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cvu1,
            cvu2,
            100,
            fake.date_this_century(),
            "Transferencia",
            "Pendiente",
            False,
            None,
            0,
        ),
    )
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

## Not enough balance

In [156]:
def create_transaccion_debit_between_users(
        sender : str,
        reciever : str,
        amount : float,
        description : str
    ) -> int:
    """Debita una cantidad de dinero de un usuario y la transfiere a otro

    Args:
        sender (str): CVU del usuario que envía el dinero
        reciever (str): CVU del usuario que recibe el dinero
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        int: Código de la transacción
    """

    # Check for balance first
    if pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (sender,)).fetchone()[0] < amount:
        raise Exception("Not enough balance")

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            sender,
            reciever,
            amount,
            fake.date_this_century(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    pg.execute(
        "UPDATE Usuarios SET saldo = saldo - %s WHERE clave_uniforme = %s",
        (amount, sender),
    )

    pg.execute(
        "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
        (amount, reciever),
    )

    return pg.execute("SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1").fetchone()[0]

# Try again to insert a transaction without enough balance

try:
    create_transaccion_debit_between_users(cvu1, cvu2, 100, "Transferencia")
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

## Not enough balance

In [157]:
# Deposit money to user 1 from a bank account

def create_transaccion_deposit(
    user: str, cbu_cuenta_bancaria: str, amount: float, description: str
) -> Tuple[str, float]:
    """Deposita dinero en la cuenta de un usuario

    Args:
        user (str): CVU del usuario
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cvu_cuenta1,
            user,
            amount,
            fake.date_this_century(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    pg.execute(
        "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
        (amount, user),
    )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )


codigo_transaccion, saldo_final = create_transaccion_deposit(
    cvu1, cvu_cuenta1, 1000, "Depósito"
)

display(Markdown(f"## Transacción de depósito\n- Código: {codigo_transaccion}\n- Saldo final: {saldo_final}"))

## Transacción de depósito
- Código: 2
- Saldo final: 1000.0

In [158]:
# Transfer money from user 1 to user 2 now that user 1 has enough balance

codigo_transaccion = create_transaccion_debit_between_users(
    cvu1, cvu2, 100, "Transferencia"
)

display(Markdown(f"## Transacción de transferencia\n- Código: {codigo_transaccion}"))

# Buscar transacciones y mostrar

pg.execute("SELECT * FROM Transaccion WHERE codigo = %s", (codigo_transaccion,)).fetchone()

## Transacción de transferencia
- Código: 3

(3, '04830279907429520135534', '05140685864015702391984', 100.0, datetime.date(2020, 1, 25), 'Transferencia', 'eye', False, None, None)

In [159]:
# Listar nombres de Servicios

nombres = pg.execute("SELECT nombre_empresa FROM ProveedorServicio").fetchall()

display(Markdown(f"## Nombres de servicios\n- {', '.join([n[0] for n in nombres])}"))

sample_nombre = random.choice(nombres)[0]

display(Markdown(f"## Nombre de servicio aleatorio\n- {sample_nombre}"))

## Nombres de servicios
- Nelson LLC

## Nombre de servicio aleatorio
- Nelson LLC

In [160]:
# Pagar un servicio

def create_transaccion_pay_service(
    user: str, service_name: str, amount: float, description: str
) -> Tuple[str, float]:
    """Paga un servicio

    Args:
        user (str): CVU del usuario
        service_name (str): Nombre del servicio
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción
    
    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    # Check for balance first
    if pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)).fetchone()[0] < amount:
        raise Exception("Not enough balance")

    # Hallar el cbu del servicio por su nombre
    cbu_servicio = pg.execute(
        "SELECT clave_uniforme FROM ProveedorServicio WHERE nombre_empresa = %s",
        (service_name,),
    ).fetchone()[0]

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            user,
            cbu_servicio,
            amount,
            fake.date_this_century(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    pg.execute(
        "UPDATE Usuarios SET saldo = saldo - %s WHERE clave_uniforme = %s",
        (amount, user),
    )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )

create_transaccion_pay_service(cvu1, sample_nombre, 100, "Pago de servicio")

('4', 800.0)

In [166]:
# Invertir plata en la fecha y generar rendimientos a futuro


def comenzar_inversion(
    cvu: str,
):
    """
    CREATE TABLE Rendimiento (
        id SERIAL PRIMARY KEY,
        fecha_pago DATE,
        comienzo_plazo DATE NOT NULL,
        fin_plazo DATE,
        TNA FLOAT,
        monto FLOAT
    );
    """
    # Se pone a invertir el monto total del usuario
    # La fecha del comienzo del plazo es ahora,
    # El fin del plazo es en 1 dia
    # La fecha de pago es el próximo dia hábil
    # La tna es entre 60 y 80
    # El monto es el saldo del usuario

    # Solo se puede tener un rendimiento en un comienzo de plazo dado
    # Si ya hay un rendimiento activo y se va a generar otro, se debe finalizar el anterior

    # Hallar el saldo del usuario
    saldo = pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu,)
    ).fetchone()[0]

    # Hallar si hay un rendimiento activo
    rendimiento_activo = pg.execute(
        "SELECT * FROM Rendimiento WHERE comienzo_plazo = CURRENT_DATE"
    ).fetchone()

    if rendimiento_activo:
        # Finalizar el rendimiento activo
        pg.execute(
            "UPDATE Rendimiento SET fin_plazo = CURRENT_DATE WHERE id = %s",
            (rendimiento_activo[0],),
        )

    # Generar un nuevo rendimiento
    tna = random.uniform(60, 80)
    monto = saldo
    fin_plazo = datetime.now() + timedelta(days=1)

    pg.execute(
        """INSERT INTO Rendimiento (comienzo_plazo, fin_plazo, TNA, monto)
        VALUES (%s, %s, %s, %s)""",
        (
            datetime.now(),
            fin_plazo,
            tna,
            monto,
        ),
    )

    # Asociar el rendimiento al usuario
    rendimiento_id = pg.execute(
        "SELECT id FROM Rendimiento ORDER BY id DESC LIMIT 1"
    ).fetchone()[0]

    pg.execute(
        "INSERT INTO RendimientoUsuario VALUES (%s, %s)",
        (cvu, rendimiento_id),
    )

    return rendimiento_id


def pagar_rendimientos_usuario(cvu: str) -> List:
    """
    Paga los rendimientos de un usuario

    Args:
        cvu (str): Clave uniforme del usuario

    Returns:
        List: Lista de los montos pagados
    """

    # Hallar los rendimientos no pagados del usuario, es decir sin fecha de pago
    # JOIN con Rendimiento para hallar los datos del rendimiento

    rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND fecha_pago IS NULL""",
        (cvu,),
    ).fetchall()

    for rendimiento in rendimientos:
        # Pagar el rendimiento
        pg.execute(
            "UPDATE Rendimiento SET fecha_pago = CURRENT_DATE WHERE id = %s",
            (rendimiento[0],),
        )

        # Añadir el rendimiento generado al saldo del usuario
        # El rendimiento es el monto * (1 + TNA / 100) * (fin_plazo - comienzo_plazo) / 365
        rendimiento_generado = (
            rendimiento[5]
            * (1 + rendimiento[4] / 100)
            * (rendimiento[3] - rendimiento[2]).days
            / 365
        )

        pg.execute(
            "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
            (rendimiento_generado, cvu),
        )

    if not rendimientos:
        return []

    # Refetch los rendimientos para mostrar by r[0] for r in rendimientos
    updated_rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND Rendimiento.id IN %s""",
        (cvu, tuple([r[0] for r in rendimientos])),
    ).fetchall()

    return updated_rendimientos

In [161]:
rendimiento_id = comenzar_inversion(cvu1)

display(Markdown(f"## Rendimiento comenzado\n- ID: {rendimiento_id}"))

rendimientos = pagar_rendimientos_usuario(cvu1)

display(
    Markdown("## Rendimientos pagados" + "\n".join([f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + rendimientos[0][4] / 100)}
""" for r in rendimientos])
    )
)

## Rendimiento comenzado
- ID: 1

## Rendimientos pagados
- ID: 1
- Fecha de pago: 2024-05-10
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 71.24817198597383
- Monto Invertido: 800.0
- Monto Pagado: 1369.9853758877907


In [162]:
# Crear un nuevo usuario 3
# depositar 1000,
# comenzar un rendimiento
# interrumpirlo transfiriendole 500 al usuario 1 (Agregar un trigger para que cuando se cree una transacción, si el usuario tiene un rendimiento activo, se finalice y se cree uno nuevo con el monto restante pero que no se pague)
# continuar el rendimiento y pagar los rendimientos

# TLDR; Hacer una transferencia con un rendimiento activo y ver que pasa


cvu3, alias3, cuit3, email3, first_name3, last_name3, user_name3, password3 = create_usuario()

# Mostrar saldo cvu3
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

create_transaccion_deposit(cvu3, cvu_cuenta1, 1000, "Depósito")

# Mostrar saldo cvu3 después del depósito
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

rendimiento_id = comenzar_inversion(cvu3)

create_transaccion_debit_between_users(cvu3, cvu1, 500, "Transferencia")
# Mostrar saldo cvu3 después de la transferencia
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

rendimientos = pagar_rendimientos_usuario(cvu3)
# Mostrar saldo cvu3 después de pagar los rendimientos
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

display(
    Markdown(
        "## Rendimientos pagados"
        + "\n".join(
            [
                f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + rendimientos[0][4] / 100)}
"""
                for r in rendimientos
            ]
        )
    )
)

0.0

1000.0

500.0

504.7050707638222

## Rendimientos pagados
- ID: 2
- Fecha de pago: 2024-05-10
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 71.73508287951043
- Monto Invertido: 1000.0
- Monto Pagado: 1717.3508287951042


In [168]:
# Obtener todos los registros de rendimientos de cvu3

pg.execute(
    """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s""",
    (cvu3,),
).fetchall()

[(2, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 71.73508287951043, 1000.0)]

In [164]:
# Mostrar usuario cvu2

pg.execute("SELECT * FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()

('0770258812130048829932', '60-81255451-0', 'katie22@example.net', 'Peter', 'Cooke', 'gvelez', '*ZTXQaiU1I', 504.7050707638222, datetime.date(2012, 7, 24))